In [1]:
# %%
import pandas as pd
import pyodbc
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

cnxn = pyodbc.connect(r'Driver=SQL Server;Server=dwhbisqltst;Database=SCM_UTE_02;Trusted_Connection=yes;')
cursor = cnxn.cursor()

sql_query1 = """
WITH Data AS (
    SELECT 
        sdhMachineId, 
        data_value, 
        CONVERT(datetime, [timestamp]) AS [from]
    FROM [SCM_INTEGRATION].[dbo].[INT_CR_Measure_hot_MachineStatus]
    WHERE [timestamp_date] > DATEADD(DAY, -365, GETDATE())
      AND [timestamp_date] <= GETDATE()
),
MinMaxDates AS (
    SELECT 
        MIN([from]) AS min_from,
        MAX([from]) AS max_from
    FROM Data
),
InitialRow AS (
    SELECT 
        DISTINCT sdhMachineId, 
        'UPDATED' AS data_value, 
        (SELECT min_from FROM MinMaxDates) AS [from]
    FROM Data
),
FinalRow AS (
    SELECT 
        DISTINCT sdhMachineId, 
        'UPDATED' AS data_value, 
        (SELECT max_from FROM MinMaxDates) AS [from]
    FROM Data
),
AllData AS (
    SELECT 
        sdhMachineId, 
        data_value, 
        [from]
    FROM Data
    UNION ALL
    SELECT 
        sdhMachineId, 
        data_value, 
        [from]
    FROM InitialRow
    UNION ALL
    SELECT 
        sdhMachineId, 
        data_value, 
        [from]
    FROM FinalRow
),
AllDataWithLag AS (
    SELECT 
        sdhMachineId, 
        data_value, 
        [from],
        LAG([from]) OVER (PARTITION BY sdhMachineId ORDER BY [from] DESC) AS [to]
    FROM AllData
),
DateSplit AS (
    SELECT
        sdhMachineId,
        data_value,
        [from],
        [to],
        CAST([from] AS DATE) AS start_date,
        CAST([to] AS DATE) AS end_date
    FROM AllDataWithLag
),
DateRange AS (
    SELECT
        sdhMachineId,
        data_value,
        [from],
        [to],
        start_date,
        end_date,
        start_date AS current_day
    FROM DateSplit
    UNION ALL
    SELECT
        sdhMachineId,
        data_value,
        [from],
        [to],
        start_date,
        end_date,
        DATEADD(DAY, 1, current_day) AS current_day
    FROM DateRange
    WHERE DATEADD(DAY, 1, current_day) <= end_date
)
SELECT
    sdhMachineId,
    data_value,
    CASE 
        WHEN current_day = start_date THEN [from]
        ELSE CAST(current_day AS DATETIME)
    END AS [from],
    CASE 
        WHEN current_day = end_date THEN [to]
        ELSE DATEADD(SECOND, -1, DATEADD(DAY, 1, CAST(current_day AS DATETIME)))
    END AS [to]
FROM DateRange
ORDER BY sdhMachineId, [from]
OPTION (MAXRECURSION 1000);  -- Aumenta il limite di ricorsione a 1000


"""

df_status = pd.read_sql(sql_query1, cnxn)



In [2]:
import pandas as pd
import os

file_path = r'C:\Users\dgarattoni\OneDrive - scmgroup.com\my ONEDRIVE\Business Analyst in Spare Parts\06.CONTROL ROOM\8. REPORTISTICA\status data\df_Status_1.0.1.csv'


df_status.to_csv(file_path, index=False)

print(f"DataFrame saved to {file_path}")

DataFrame saved to C:\Users\dgarattoni\OneDrive - scmgroup.com\my ONEDRIVE\Business Analyst in Spare Parts\06.CONTROL ROOM\8. REPORTISTICA\status data\df_Status_1.0.1.csv
